# W3 - Graphs & Fields

In this workshop we will learn about graphs & fields, and explore their application in configuration design.

## 0. Initialization

## 1. Graphs

### 1.1. Spatial Relations

## 2. Fields

### 2.1. Spatial Data

## Credits

In [ ]:
__author__ = "Shervin Azadi"
__license__ = "MIT"
__version__ = "1.0"
__url__ = "https://github.com/shervinazadi/earthy_workshops"
__summary__ = "Earthy Design Studio"